In [ ]:
#Esto es un cuaderno, para realizar el proyecto paso a paso

Importamos librerias necesarias en la limpieza de datos (no olvidar instalar las libreras necesarias con pip install .. antes de importarlas)

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OrdinalEncoder

import folium
import plotly.express as px


Leemos el primer csv, y lo asignamos como nuestro dataframe. Imprimimos las primeras lineas para tener una idea del dataframe

In [7]:
df_ca=pd.read_csv("commercial_activity_df.csv")
print(df_ca.head())

   Unnamed: 0   pk_cid pk_partition  entry_date entry_channel  \
0           0  1375586   2018-01-28  2018-01-12           KHL   
1           1  1050611   2018-01-28  2015-08-10           KHE   
2           2  1050612   2018-01-28  2015-08-10           KHE   
3           3  1050613   2018-01-28  2015-08-10           KHD   
4           4  1050614   2018-01-28  2015-08-10           KHE   

   active_customer             segment  
0              1.0   02 - PARTICULARES  
1              0.0  03 - UNIVERSITARIO  
2              0.0  03 - UNIVERSITARIO  
3              0.0  03 - UNIVERSITARIO  
4              1.0  03 - UNIVERSITARIO  


Obtenemos informacion general del dataframe con  los metodos .describe() e .info()

In [9]:
df_ca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   pk_cid           int64  
 2   pk_partition     object 
 3   entry_date       object 
 4   entry_channel    object 
 5   active_customer  float64
 6   segment          object 
dtypes: float64(1), int64(2), object(4)
memory usage: 318.5+ MB


Vemos que existe una columna de numeros, que coincide con el numero de los registros, pero que no es el indice, llamada Unnamed. Vamos a cambiar el nombre a esta columna y asignarla como indice.

In [10]:
n_index = 'Index'
df_ca = df_ca.rename_axis(n_index).reset_index()

In [11]:
df_ca.set_index('Index', inplace=True)

Comprobamos que el indice ahora es correcto, pero nos sobra la columna Unnamed, la borramos con del

In [15]:
print(df_ca.head())

        pk_cid pk_partition  entry_date entry_channel  active_customer  \
Index                                                                    
0      1375586   2018-01-28  2018-01-12           KHL              1.0   
1      1050611   2018-01-28  2015-08-10           KHE              0.0   
2      1050612   2018-01-28  2015-08-10           KHE              0.0   
3      1050613   2018-01-28  2015-08-10           KHD              0.0   
4      1050614   2018-01-28  2015-08-10           KHE              1.0   

                  segment  
Index                      
0       02 - PARTICULARES  
1      03 - UNIVERSITARIO  
2      03 - UNIVERSITARIO  
3      03 - UNIVERSITARIO  
4      03 - UNIVERSITARIO  


In [14]:
del(df_ca["Unnamed: 0"])

In [16]:
print(df_ca.head())

        pk_cid pk_partition  entry_date entry_channel  active_customer  \
Index                                                                    
0      1375586   2018-01-28  2018-01-12           KHL              1.0   
1      1050611   2018-01-28  2015-08-10           KHE              0.0   
2      1050612   2018-01-28  2015-08-10           KHE              0.0   
3      1050613   2018-01-28  2015-08-10           KHD              0.0   
4      1050614   2018-01-28  2015-08-10           KHE              1.0   

                  segment  
Index                      
0       02 - PARTICULARES  
1      03 - UNIVERSITARIO  
2      03 - UNIVERSITARIO  
3      03 - UNIVERSITARIO  
4      03 - UNIVERSITARIO  
